# Day 9

There's distress signal coming in! But in order to hear it, we have to boost our sensors. Luckily we have a computer to do that with

## Part 1

In order to handle to boost program inputs, we need to handle a new parameter mode, a new opcode, and improved memory

Given the BOOST program input, produce the appropriate BOOST key code (output) to prove that our computer can handle the boost program

In [1]:
import math

from itertools import permutations
from typing import Tuple, List, Union

In [8]:
class Computer:
    def __init__(self, memory: List[int], pointer: int = 0, inputs: List[int] = None):
        self.memory = memory.copy()
        self.pointer = pointer
        self.relative_base = 0
        self.inputs = inputs if inputs else []
        self.outputs = []
        self.complete = False
        self.pointer_increment_dict = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2}
    
    @staticmethod
    def parse_opcode(opcode: int) -> Tuple[int]:
        opcode_param_list = [int(num) for num in str(opcode).zfill(5)]
        # Don't need the leading zero from the opcode
        return tuple(opcode_param_list[:3] + opcode_param_list[4:])
    
    def get_output_positions(self, first_param_mode: int, second_param_mode: int, third_param_mode) -> Tuple[int]:
        if first_param_mode == 1:
            first_output_position = self.pointer + 1
        elif first_param_mode == 2:
            first_output_position = self.memory[self.pointer + 1] + self.relative_base
        else:
            first_output_position = self.memory[self.pointer + 1]
    
        try:
            if second_param_mode == 1:
                second_output_position = self.pointer + 2
            elif second_param_mode == 2:
                second_output_position = self.memory[self.pointer + 2] + self.relative_base
            else:
                second_output_position = self.memory[self.pointer + 2]
        except IndexError:
            return (first_output_position, 0)
        
        try:
            if third_param_mode == 1:
                third_output_position = self.pointer + 3
            elif third_param_mode == 2:
                third_output_position = self.memory[self.pointer + 3] + self.relative_base
            else:
                third_output_position = self.memory[self.pointer + 3]
        except IndexError:
            return (first_output_position, second_output_position, 0)

        return (first_output_position, second_output_position, third_output_position)
    
    def run_instruction(self, opcode: int, debug: bool = False):
        # Third param mode is always in position mode
        third_param_mode, second_param_mode, first_param_mode, opcode = self.parse_opcode(opcode)
        first_output_position, second_output_position, third_output_position = self.get_output_positions(
            first_param_mode, second_param_mode, third_param_mode
        )
        
        if opcode not in self.pointer_increment_dict.keys():
            raise ValueError("Invalid opcode!")
        
        # For opcodes that have a third parameter
        if opcode in [1, 2, 7, 8]:
            try:
                self.memory[third_output_position]
            except IndexError:
                additional_memory = third_output_position + 1 - len(self.memory)
                if debug:
                    print(f"Adding {additional_memory} more memory")
                self.memory += [0 for _ in range(additional_memory)]
                        
        if opcode == 4:
            if debug:
                print(f"Adding ouput: {self.memory[first_output_position]} to outputs: {self.outputs}")
            self.outputs += [self.memory[first_output_position]]
        elif opcode == 5 and self.memory[first_output_position] != 0:
            if debug:
                print(f"Setting address from {self.pointer} to {self.memory[second_output_position]}")
            self.pointer = self.memory[second_output_position]
            # Early return so we don't change the pointer at the end of the fx
            return
        elif opcode == 6 and self.memory[first_output_position] == 0:
            if debug:
                print(f"Setting address from {self.pointer} to {self.memory[second_output_position]}")
            self.pointer = self.memory[second_output_position]
            # Early return so we don't change the pointer at the end of the fx
            return

        if opcode == 1:
            if debug:
                print(f"Replacing {self.memory[third_output_position]} at address {third_output_position} with {self.memory[first_output_position] + self.memory[second_output_position]}")
            self.memory[third_output_position] = self.memory[first_output_position] + self.memory[second_output_position]
        elif opcode == 2:
            if debug:
                print(f"Replacing {self.memory[third_output_position]} at address {third_output_position} with {self.memory[first_output_position] * self.memory[second_output_position]}")
            self.memory[third_output_position] = self.memory[first_output_position] * self.memory[second_output_position]
        elif opcode == 3:
            if debug:
                print(f"Using next input from: {self.inputs}")
            computer_input = self.inputs.pop(0)
            if debug:
                print(f"Replacing {self.memory[first_output_position]} at {first_output_position} with {computer_input}")
            self.memory[first_output_position] = computer_input
        elif opcode == 7:
            replacement = 1 if self.memory[first_output_position] < self.memory[second_output_position] else 0
            if debug:
                print(f"Replacing {self.memory[third_output_position]} at address {third_output_position} with {replacement}")
            self.memory[third_output_position] = replacement
        elif opcode == 8:
            replacement = 1 if self.memory[first_output_position] == self.memory[second_output_position] else 0
            if debug:
                print(f"Replacing {self.memory[third_output_position]} at address {third_output_position} with {replacement}")
            self.memory[third_output_position] = replacement
        elif opcode == 9:
            relative_base_adj = self.memory[first_output_position]
            if debug:
                print(f"Adjusting relative base from {self.relative_base} to {self.relative_base + relative_base_adj}")
            self.relative_base += relative_base_adj
        
        if debug:
            print(f"Setting address from {self.pointer} to {self.pointer + self.pointer_increment_dict.get(opcode)}")
        # Increment pointer
        self.pointer += self.pointer_increment_dict.get(opcode)
    
    def run_program(self, debug=False):
        while not self.complete:
            opcode = self.memory[self.pointer]
            if debug:
                print(f"opcode: {opcode} at address: {self.pointer}")
                #print(f"relative memory: {self.memory[self.pointer:self.pointer + 4]}")
                print(f"memory before instruction: {self.memory}")
            
            if opcode == 99:
                self.complete = True
            elif str(opcode).endswith("3") and not self.inputs:
                if debug:
                    print(f"No more inputs, exiting")
                break
            else:
                self.run_instruction(opcode, debug)
            
            if debug:
                print(f"memory after instruction: {self.memory}")

In [3]:
program = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
computer = Computer(program)
computer.run_program()
assert computer.outputs == program

In [4]:
program = [1102,34915192,34915192,7,4,7,99,0]
computer = Computer(program)
computer.run_program()
assert computer.outputs[0] == 1219070632396864

In [5]:
program = [104,1125899906842624,99]
computer = Computer(program)
computer.run_program()
assert computer.outputs[0] == program[1]

In [10]:
# Puzzle input
boost_program = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1101,3,0,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,902,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1101,26,0,1015,1101,29,0,1010,1102,1,24,1013,1102,1,33,1008,1102,36,1,1012,1101,0,572,1023,1101,35,0,1014,1101,0,38,1019,1102,1,30,1006,1101,0,890,1029,1101,34,0,1011,1101,28,0,1002,1102,1,1,1021,1101,0,37,1001,1101,0,197,1026,1101,22,0,1017,1102,1,895,1028,1101,0,20,1007,1102,21,1,1004,1102,1,39,1016,1101,0,0,1020,1102,1,190,1027,1101,0,775,1024,1102,31,1,1018,1101,0,23,1003,1101,0,25,1009,1101,770,0,1025,1101,0,27,1000,1102,1,575,1022,1101,0,32,1005,109,27,2106,0,0,1001,64,1,64,1106,0,199,4,187,1002,64,2,64,109,-18,21101,40,0,5,1008,1014,39,63,1005,63,219,1106,0,225,4,205,1001,64,1,64,1002,64,2,64,109,-6,1201,-1,0,63,1008,63,28,63,1005,63,251,4,231,1001,64,1,64,1105,1,251,1002,64,2,64,109,5,21102,41,1,3,1008,1011,38,63,1005,63,271,1105,1,277,4,257,1001,64,1,64,1002,64,2,64,109,-7,2102,1,1,63,1008,63,28,63,1005,63,299,4,283,1106,0,303,1001,64,1,64,1002,64,2,64,109,-7,1207,10,22,63,1005,63,321,4,309,1106,0,325,1001,64,1,64,1002,64,2,64,109,16,2107,31,-4,63,1005,63,345,1001,64,1,64,1105,1,347,4,331,1002,64,2,64,109,-9,1201,3,0,63,1008,63,18,63,1005,63,371,1001,64,1,64,1106,0,373,4,353,1002,64,2,64,109,7,1202,-7,1,63,1008,63,40,63,1005,63,393,1106,0,399,4,379,1001,64,1,64,1002,64,2,64,109,-5,1208,5,33,63,1005,63,417,4,405,1106,0,421,1001,64,1,64,1002,64,2,64,109,1,1202,2,1,63,1008,63,30,63,1005,63,443,4,427,1105,1,447,1001,64,1,64,1002,64,2,64,109,-7,2102,1,10,63,1008,63,19,63,1005,63,471,1001,64,1,64,1105,1,473,4,453,1002,64,2,64,109,6,2108,21,0,63,1005,63,489,1105,1,495,4,479,1001,64,1,64,1002,64,2,64,109,9,21108,42,42,0,1005,1012,513,4,501,1105,1,517,1001,64,1,64,1002,64,2,64,109,7,21107,43,44,-1,1005,1018,535,4,523,1106,0,539,1001,64,1,64,1002,64,2,64,109,-5,21101,44,0,2,1008,1016,44,63,1005,63,561,4,545,1105,1,565,1001,64,1,64,1002,64,2,64,2105,1,9,1106,0,581,4,569,1001,64,1,64,1002,64,2,64,109,13,21107,45,44,-9,1005,1018,597,1105,1,603,4,587,1001,64,1,64,1002,64,2,64,109,-25,2101,0,3,63,1008,63,32,63,1005,63,625,4,609,1105,1,629,1001,64,1,64,1002,64,2,64,109,7,1208,-7,30,63,1005,63,645,1105,1,651,4,635,1001,64,1,64,1002,64,2,64,109,-2,21102,46,1,9,1008,1016,46,63,1005,63,677,4,657,1001,64,1,64,1106,0,677,1002,64,2,64,109,-2,21108,47,48,9,1005,1014,697,1001,64,1,64,1105,1,699,4,683,1002,64,2,64,109,14,1205,2,713,4,705,1105,1,717,1001,64,1,64,1002,64,2,64,109,-7,1206,8,735,4,723,1001,64,1,64,1106,0,735,1002,64,2,64,109,-18,2101,0,6,63,1008,63,24,63,1005,63,759,1001,64,1,64,1106,0,761,4,741,1002,64,2,64,109,29,2105,1,1,4,767,1106,0,779,1001,64,1,64,1002,64,2,64,109,-5,1206,3,791,1106,0,797,4,785,1001,64,1,64,1002,64,2,64,109,-12,2107,31,-1,63,1005,63,819,4,803,1001,64,1,64,1105,1,819,1002,64,2,64,109,7,1205,7,835,1001,64,1,64,1105,1,837,4,825,1002,64,2,64,109,-11,1207,7,24,63,1005,63,853,1106,0,859,4,843,1001,64,1,64,1002,64,2,64,109,4,2108,27,-6,63,1005,63,881,4,865,1001,64,1,64,1106,0,881,1002,64,2,64,109,24,2106,0,-2,4,887,1106,0,899,1001,64,1,64,4,64,99,21102,27,1,1,21101,0,913,0,1106,0,920,21201,1,61934,1,204,1,99,109,3,1207,-2,3,63,1005,63,962,21201,-2,-1,1,21101,0,940,0,1106,0,920,21202,1,1,-1,21201,-2,-3,1,21101,0,955,0,1105,1,920,22201,1,-1,-2,1105,1,966,22102,1,-2,-2,109,-3,2105,1,0]
computer = Computer(boost_program, inputs=[1])
computer.run_program()
assert computer.outputs == [2406950601]

In [11]:
# Answer
computer.outputs[0]

2406950601

# Part 2

We now have a fully functioning intcode computer :tada:

Now we need to run the boost program with a input of 2 to find the coordinates for the distress signal

In [12]:
boost_program = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1101,3,0,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,902,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1101,26,0,1015,1101,29,0,1010,1102,1,24,1013,1102,1,33,1008,1102,36,1,1012,1101,0,572,1023,1101,35,0,1014,1101,0,38,1019,1102,1,30,1006,1101,0,890,1029,1101,34,0,1011,1101,28,0,1002,1102,1,1,1021,1101,0,37,1001,1101,0,197,1026,1101,22,0,1017,1102,1,895,1028,1101,0,20,1007,1102,21,1,1004,1102,1,39,1016,1101,0,0,1020,1102,1,190,1027,1101,0,775,1024,1102,31,1,1018,1101,0,23,1003,1101,0,25,1009,1101,770,0,1025,1101,0,27,1000,1102,1,575,1022,1101,0,32,1005,109,27,2106,0,0,1001,64,1,64,1106,0,199,4,187,1002,64,2,64,109,-18,21101,40,0,5,1008,1014,39,63,1005,63,219,1106,0,225,4,205,1001,64,1,64,1002,64,2,64,109,-6,1201,-1,0,63,1008,63,28,63,1005,63,251,4,231,1001,64,1,64,1105,1,251,1002,64,2,64,109,5,21102,41,1,3,1008,1011,38,63,1005,63,271,1105,1,277,4,257,1001,64,1,64,1002,64,2,64,109,-7,2102,1,1,63,1008,63,28,63,1005,63,299,4,283,1106,0,303,1001,64,1,64,1002,64,2,64,109,-7,1207,10,22,63,1005,63,321,4,309,1106,0,325,1001,64,1,64,1002,64,2,64,109,16,2107,31,-4,63,1005,63,345,1001,64,1,64,1105,1,347,4,331,1002,64,2,64,109,-9,1201,3,0,63,1008,63,18,63,1005,63,371,1001,64,1,64,1106,0,373,4,353,1002,64,2,64,109,7,1202,-7,1,63,1008,63,40,63,1005,63,393,1106,0,399,4,379,1001,64,1,64,1002,64,2,64,109,-5,1208,5,33,63,1005,63,417,4,405,1106,0,421,1001,64,1,64,1002,64,2,64,109,1,1202,2,1,63,1008,63,30,63,1005,63,443,4,427,1105,1,447,1001,64,1,64,1002,64,2,64,109,-7,2102,1,10,63,1008,63,19,63,1005,63,471,1001,64,1,64,1105,1,473,4,453,1002,64,2,64,109,6,2108,21,0,63,1005,63,489,1105,1,495,4,479,1001,64,1,64,1002,64,2,64,109,9,21108,42,42,0,1005,1012,513,4,501,1105,1,517,1001,64,1,64,1002,64,2,64,109,7,21107,43,44,-1,1005,1018,535,4,523,1106,0,539,1001,64,1,64,1002,64,2,64,109,-5,21101,44,0,2,1008,1016,44,63,1005,63,561,4,545,1105,1,565,1001,64,1,64,1002,64,2,64,2105,1,9,1106,0,581,4,569,1001,64,1,64,1002,64,2,64,109,13,21107,45,44,-9,1005,1018,597,1105,1,603,4,587,1001,64,1,64,1002,64,2,64,109,-25,2101,0,3,63,1008,63,32,63,1005,63,625,4,609,1105,1,629,1001,64,1,64,1002,64,2,64,109,7,1208,-7,30,63,1005,63,645,1105,1,651,4,635,1001,64,1,64,1002,64,2,64,109,-2,21102,46,1,9,1008,1016,46,63,1005,63,677,4,657,1001,64,1,64,1106,0,677,1002,64,2,64,109,-2,21108,47,48,9,1005,1014,697,1001,64,1,64,1105,1,699,4,683,1002,64,2,64,109,14,1205,2,713,4,705,1105,1,717,1001,64,1,64,1002,64,2,64,109,-7,1206,8,735,4,723,1001,64,1,64,1106,0,735,1002,64,2,64,109,-18,2101,0,6,63,1008,63,24,63,1005,63,759,1001,64,1,64,1106,0,761,4,741,1002,64,2,64,109,29,2105,1,1,4,767,1106,0,779,1001,64,1,64,1002,64,2,64,109,-5,1206,3,791,1106,0,797,4,785,1001,64,1,64,1002,64,2,64,109,-12,2107,31,-1,63,1005,63,819,4,803,1001,64,1,64,1105,1,819,1002,64,2,64,109,7,1205,7,835,1001,64,1,64,1105,1,837,4,825,1002,64,2,64,109,-11,1207,7,24,63,1005,63,853,1106,0,859,4,843,1001,64,1,64,1002,64,2,64,109,4,2108,27,-6,63,1005,63,881,4,865,1001,64,1,64,1106,0,881,1002,64,2,64,109,24,2106,0,-2,4,887,1106,0,899,1001,64,1,64,4,64,99,21102,27,1,1,21101,0,913,0,1106,0,920,21201,1,61934,1,204,1,99,109,3,1207,-2,3,63,1005,63,962,21201,-2,-1,1,21101,0,940,0,1106,0,920,21202,1,1,-1,21201,-2,-3,1,21101,0,955,0,1105,1,920,22201,1,-1,-2,1105,1,966,22102,1,-2,-2,109,-3,2105,1,0]
computer = Computer(boost_program, inputs=[2])
computer.run_program()
computer.outputs

[83239]